In [15]:
import os
from omdb_tools import *
from dotenv import load_dotenv
import pprint
##to load credentials
load_dotenv()

True

### Set all the keys

In [16]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_HUB_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") ## Put your Langsmith API key here
os.environ["LANGCHAIN_TRACING_V2"] = 'true' ## Set this as True
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com/' ## Set this as: https://api.smith.langchain.com/
os.environ["LANGCHAIN_HUB_API_URL"] = 'https://api.hub.langchain.com' ## Set this as : https://api.hub.langchain.com
os.environ["LANGCHAIN_PROJECT"] = 'llm-agents'

### Define your tools

In [17]:
### Define your tools
movie_info_tool = OmdbAPI.get_movie_info_by_title
search_movie_tool = OmdbAPI.search_movie
search_series_tool = OmdbAPI.search_series

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [movie_info_tool, search_movie_tool, search_series_tool, wikipedia]

### Test the tools

In [18]:
movie_title_tool = OmdbAPI.get_movie_info_by_title
print(movie_title_tool.name, movie_title_tool.description)

	
data = 'Harry%Potter|2001'
tool_output = movie_title_tool.run(data)
pprint.pprint(tool_output)

Movie_Info_By_Title_Year Call this tool to get information about a movie if you have its title. Optionally you can also specify a movie's year.
       The input to this tool is a movie title. Year can also be optionally specified. If year is specified it must be separated by pipe
       Example if the user is look True Grit then input should be `True%20Grit`. Represent space as %20
       Example if the user is look True Grit movie from 1969 then input should be `True%20Grit|1969`. Represent space as %20. Separate year by |
       The output from this tool will be information on title, plot, director and tomato meter rating of the movie that best matches
{'actors': 'Daniel Radcliffe, Rupert Grint, Emma Watson',
 'awards': 'Nominated for 3 Oscars. 20 wins & 74 nominations total',
 'box_office': '$318,886,962',
 'country': 'United Kingdom, United States',
 'director': 'Chris Columbus',
 'dvd': 'N/A',
 'genre': 'Adventure, Family, Fantasy',
 'imdb_id': 'tt0241527',
 'imdb_rating': '7.6',


In [19]:
search_movie_tool = OmdbAPI.search_movie

print(search_movie_tool.name, search_movie_tool.description)
	
data = 'Naruto'
tool_output = search_movie_tool.run(data)
pprint.pprint(tool_output)

Search_Movie Call this tool to search for a movie based on search string. The input to the tool is a search string.
       The output from the tool is all the results that match the search criteria
       Example if the user is look for a movie Naruto, then data=Naruto
[{'imdb_id': 'tt3717532',
  'poster': 'https://m.media-amazon.com/images/M/MV5BMjk1NzA4Njg4Ml5BMl5BanBnXkFtZTgwMDgxODQ5MzE@._V1_SX300.jpg',
  'title': 'The Last: Naruto the Movie',
  'type': 'movie',
  'year': '2014'},
 {'imdb_id': 'tt4618398',
  'poster': 'https://m.media-amazon.com/images/M/MV5BNDIwYjg5YzItZTJmMC00YjJhLWFiNjYtMDU5MmQxOTQxODM2XkEyXkFqcGdeQXVyMjc2Nzg5OTQ@._V1_SX300.jpg',
  'title': 'Boruto: Naruto the Movie',
  'type': 'movie',
  'year': '2015'},
 {'imdb_id': 'tt2290828',
  'poster': 'https://m.media-amazon.com/images/M/MV5BMTQ5MTcyNDYwMV5BMl5BanBnXkFtZTgwNzMzNzc0MjE@._V1_SX300.jpg',
  'title': 'Road to Ninja - Naruto the Movie',
  'type': 'movie',
  'year': '2012'},
 {'imdb_id': 'tt0476680',
  'poster':

In [20]:
wikipedia.name, wikipedia.description

('wikipedia',
 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.')

In [21]:
tool_output = wikipedia.run("Boruto: Naruto the Movie")
pprint.pprint(tool_output)

('Page: Boruto: Naruto the Movie\n'
 'Summary: Boruto: Naruto the Movie is a 2015 Japanese animated  martial arts '
 'fantasy film and the directorial debut of Hiroyuki Yamashita. It is based on '
 "Masashi Kishimoto's manga and anime Naruto, and is the second film to be a "
 'part of the canonical Naruto storyline, being a sequel to the manga series. '
 'It stars Yūko Sanpei, Junko Takeuchi, Kokoro Kikuchi and Noriaki Sugiyama. '
 'Set after the finale of Naruto, the film focuses on the title character '
 "Boruto Uzumaki, son of  Naruto's protagonist, Naruto Uzumaki, who cannot "
 'stay with his family due to being the leader of his ninja village. Vigilante '
 'Sasuke Uchiha returns to the village with warnings about two beings who '
 'might become a big threat to the world peace the ninjas managed to bring '
 'about in the original series.\n'
 'The film was first teased in the post-credits scene of the previous film, '
 'The Last: Naruto the Movie (2014). Kishimoto took a large role 

### Define a Groq Llama3 agent

Code and idea borrowed from: https://www.youtube.com/watch?v=j2OAeeujQ9M&t=937s

In [22]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.messages import SystemMessage

In [23]:
# Prompt
system_prompt = '''You are a helpful assistant who helps users get information
about movies and TV series. You can use a few different tools you have access to:

1. movie_info_tool - The input to this tool is movie title and optionally year. The
tool returns a brief plot, tomato meter rating and other information for the movie that matches the title

2. search_movie_tool - Use this tool to search for all movies matching a search string.

3. search_series_tool - Use this tool to search for all TV series matching a search string.

4. wikipedia - Generic tool to search for anything in the vast wikipedia corpus. Use this to
search and return information on any movie/TV series related question

You can use these tools in conjunction. For example if the user wants information of the 
earliest Harry Potter movie, then use the search_movie_tool to get information of all Harry 
Potter movies and then use the movie_info_tool to get details of the first movie by
searching based on its title.

Your job is to be a conversational Movie Asssistant who keeps conversation going. You are witty and smart
and answer user's questions, plus ask your own to keep it lively.
If the user conversation is not related to any of the tools you have then answer to the best of your knowledge
For a generic question, not related to movies/TV series feel free to use the wikipedia tool

Look at the chain of messages below that include the user request.
User Message : {input}

Prior Chat history : {chat_history}
'''

assistant_prompt = PromptTemplate(
        input_variables=["input", "chat_history"], template=system_prompt)


# LLM
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.3)
# LLM Chain
assistant_agent = assistant_prompt | llm.bind_tools(tools)

In [24]:
input = "Tell me about the first harry potter movie"
chat_history = []

results = assistant_agent.invoke({"input": input, "chat_history": chat_history})
print(results.tool_calls)

[{'name': 'Search_Movie', 'args': {'data': 'Harry Potter'}, 'id': 'call_rqgm'}]


In [25]:
input = "I need information on the movie called Inception including its tomato meter rating"
chat_history = []

results = assistant_agent.invoke({"input": input, "chat_history": chat_history})
print(results.tool_calls)

[{'name': 'Movie_Info_By_Title_Year', 'args': {'data': 'Inception'}, 'id': 'call_te08'}]


In [26]:
input = "Tell me the first president of India"
chat_history = []

results = assistant_agent.invoke({"input": input, "chat_history": chat_history})
print(results.tool_calls)

[{'name': 'wikipedia', 'args': {'query': 'first president of India'}, 'id': 'call_d9dc'}]


### Put this as a Class

In [27]:
class Chatbot:
    def __init__(self, model='llama3-70b-8192', temperature=0.3):
        self.llm =  ChatGroq(model_name=model, temperature=temperature)
        self.system_prompt = '''You are a helpful assistant who helps users get information
        about movies and TV series. You can use a few different tools you have access to:

        1. movie_info_tool - The input to this tool is movie title and optionally year. The
        tool returns a brief plot, tomato meter rating and other information for the movie that matches the title

        2. search_movie_tool - Use this tool to search for all movies matching a search string.

        3. search_series_tool - Use this tool to search for all TV series matching a search string.

        4. wikipedia - Generic tool to search for anything in the vast wikipedia corpus. Use this to
        search and return information on any movie/TV series related question

        You can use these tools in conjunction. For example if the user wants information of the 
        earliest Harry Potter movie, then use the search_movie_tool to get information of all Harry 
        Potter movies and then use the movie_info_tool to get details of the first movie by
        searching based on its title.

        Your job is to be a conversational Movie Asssistant who keeps conversation going. You are witty and smart
        and answer user's questions, plus ask your own to keep it lively.
        If the user conversation is not related to any of the tools you have then answer to the best of your knowledge
        For a generic question, not related to movies/TV series feel free to use the wikipedia tool

        Look at the user message below including the message history chain.
        '''
        self.memory = ChatMessageHistory(session_id="test-session")
        self.tools = [movie_info_tool, search_movie_tool, search_series_tool, wikipedia]
    
    def run(self, input):
        assistant_prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
            content=self.system_prompt
        ),
          MessagesPlaceholder(variable_name="chat_history"),   
          MessagesPlaceholder(variable_name="agent_scratchpad"),

         ("user", "{input}"),            
        ] )

        # Create the agent and its executor
        assistant_agent = create_openai_tools_agent(self.llm, self.tools, assistant_prompt)
        agent_executor = AgentExecutor(agent=assistant_agent, tools=tools, verbose=True)

        agent_with_chat_history = RunnableWithMessageHistory(
            agent_executor,
            # This is needed because in most real world scenarios, a session id is needed
            # It isn't really used here because we are using a simple in memory ChatMessageHistory
            lambda session_id: self.memory,
            input_messages_key="input",
            history_messages_key="chat_history",
        )
        result = agent_with_chat_history.invoke({'input': input,}, config={"configurable": {"session_id": "1234"}})
        return result

In [28]:
movie_agent = Chatbot(model='llama3-8b-8192')
print(movie_agent.run("Tell me about the movie inception"))



> Entering new AgentExecutor chain...

Invoking: `Movie_Info_By_Title_Year` with `{'data': 'Inception'}`


{'title': 'Inception', 'year': '2010', 'rated': 'PG-13', 'released': '16 Jul 2010', 'runtime': '148 min', 'genre': 'Action, Adventure, Sci-Fi', 'director': 'Christopher Nolan', 'writer': 'Christopher Nolan', 'actors': 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page', 'plot': "Dom Cobb is a skilled thief, the absolute best in the dangerous art of extraction, stealing valuable secrets from deep within the subconscious during the dream state, when the mind is at its most vulnerable. Cobb's rare ability has made him a coveted player in this treacherous new world of corporate espionage, but it has also made him an international fugitive and cost him everything he has ever loved. Now Cobb is being offered a chance at redemption. One last job could give him his life back but only if he can accomplish the impossible, inception. Instead of the perfect heist, Cobb and his team of sp